In [1]:
from __future__ import print_function

from miscc.config import cfg, cfg_from_file
from datasets import TextDataset
from trainer import condGANTrainer as trainer

import os
import sys
import time
import random
import pprint
import datetime
import dateutil.tz
import argparse
import numpy as np

import torch
import torchvision.transforms as transforms

# dir_path = (os.path.abspath(os.path.join(os.path.realpath(__file__), './.')))
# sys.path.append(dir_path)

In [2]:
class parse_args():
    cfg_file='../code/cfg/bird_attn2.yml'
    gpu_id=3
    data_dir='../data/birds/'
    manualSeed=1
args = parse_args()

In [3]:

args = parse_args()
if args.cfg_file is not None:
    cfg_from_file(args.cfg_file)

if args.gpu_id == -1:
    cfg.CUDA = False
else:
    cfg.GPU_ID = args.gpu_id

if args.data_dir != '':
    cfg.DATA_DIR = args.data_dir
    

print('Using config:')
pprint.pprint(cfg)

if not cfg.TRAIN.FLAG:
    args.manualSeed = 100
elif args.manualSeed is None:
    args.manualSeed = random.randint(1, 10000)
random.seed(args.manualSeed)
np.random.seed(args.manualSeed)
torch.manual_seed(args.manualSeed)
if cfg.CUDA:
    torch.cuda.manual_seed_all(args.manualSeed)



Using config:
{'B_VALIDATION': False,
 'CONFIG_NAME': 'attn2',
 'CUDA': True,
 'DATASET_NAME': 'birds',
 'DATA_DIR': '../data/birds/',
 'GAN': {'B_ATTENTION': True,
         'B_DCGAN': False,
         'CONDITION_DIM': 100,
         'DF_DIM': 64,
         'GF_DIM': 32,
         'R_NUM': 2,
         'Z_DIM': 100},
 'GPU_ID': 3,
 'RNN_TYPE': 'LSTM',
 'TEXT': {'CAPTIONS_PER_IMAGE': 10, 'EMBEDDING_DIM': 256, 'WORDS_NUM': 18},
 'TRAIN': {'BATCH_SIZE': 20,
           'B_NET_D': True,
           'DISCRIMINATOR_LR': 0.0002,
           'ENCODER_LR': 0.0002,
           'FLAG': True,
           'GENERATOR_LR': 0.0002,
           'MAX_EPOCH': 600,
           'NET_E': '../DAMSMencoders/bird/text_encoder50.pth',
           'NET_G': '',
           'RNN_GRAD_CLIP': 0.25,
           'SMOOTH': {'GAMMA1': 4.0,
                      'GAMMA2': 5.0,
                      'GAMMA3': 10.0,
                      'LAMBDA': 5.0},
           'SNAPSHOT_INTERVAL': 50},
 'TREE': {'BASE_SIZE': 64, 'BRANCH_NUM': 3},
 'W

/media/MyDataStor1/mmrl/MMRL/notebooks/miscc/config.py:104: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_cfg = edict(yaml.load(f))


In [4]:
now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
output_dir = '../output/{0}_{1}_{2}'.format(cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)

split_dir, bshuffle = 'train', True
if not cfg.TRAIN.FLAG:
    # bshuffle = False
    split_dir = 'test'

In [5]:
# Get data loader
imsize = cfg.TREE.BASE_SIZE * (2 ** (cfg.TREE.BRANCH_NUM - 1))
image_transform = transforms.Compose([
    transforms.Scale(int(imsize * 76 / 64)),
    transforms.RandomCrop(imsize),
    transforms.RandomHorizontalFlip()])
dataset = TextDataset(cfg.DATA_DIR, split_dir,
                      base_size=cfg.TREE.BASE_SIZE,
                      transform=image_transform)
assert dataset

/home/mshaikh2/anaconda3/envs/pytorch-gpu/lib/python3.7/site-packages/torchvision/transforms/transforms.py:210: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


Total filenames:  11788 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg
Load filenames from: ../data/birds//train/filenames.pickle (8855)
Load filenames from: ../data/birds//test/filenames.pickle (2933)
Load from:  ../data/birds/captions.pickle
../data/birds/train
../data/birds/train/class_info.pickle


In [6]:
dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=cfg.TRAIN.BATCH_SIZE,
        drop_last=True, shuffle=bshuffle, num_workers=int(cfg.WORKERS))

# Define models and go to train/evaluate
algo = trainer(output_dir, dataloader, dataset.n_words, dataset.ixtoword)


In [7]:
start_t = time.time()
if cfg.TRAIN.FLAG:
    algo.train()
else:
    '''generate images from pre-extracted embeddings'''
    if cfg.B_VALIDATION:
        algo.sampling(split_dir)  # generate images for the whole valid dataset
    else:
        gen_example(dataset.wordtoix, algo)  # generate images for customized captions
end_t = time.time()
print('Total time for training:', end_t - start_t)


Load pretrained model from  https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth
Load image encoder from: ../DAMSMencoders/bird/image_encoder50.pth
Load text encoder from: ../DAMSMencoders/bird/text_encoder50.pth


/home/mshaikh2/anaconda3/envs/pytorch-gpu/lib/python3.7/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


# of netsD 3


/home/mshaikh2/anaconda3/envs/pytorch-gpu/lib/python3.7/site-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/mshaikh2/anaconda3/envs/pytorch-gpu/lib/python3.7/site-packages/torch/nn/functional.py:2479: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


errD0: 0.55 errD1: 0.61 errD2: 1.17 
g_loss0: 8.40 g_loss1: 10.51 g_loss2: 1.99 w_loss: 32.68 s_loss: 30.01 kl_loss: 0.01 
errD0: 0.74 errD1: 0.76 errD2: 0.99 
g_loss0: 8.94 g_loss1: 12.08 g_loss2: 2.39 w_loss: 23.33 s_loss: 25.20 kl_loss: 0.01 
errD0: 0.24 errD1: 0.24 errD2: 0.47 
g_loss0: 8.08 g_loss1: 16.08 g_loss2: 7.04 w_loss: 17.14 s_loss: 18.63 kl_loss: 0.01 
errD0: 0.35 errD1: 0.61 errD2: 0.62 
g_loss0: 8.16 g_loss1: 18.69 g_loss2: 9.09 w_loss: 20.21 s_loss: 19.19 kl_loss: 0.01 


IndexError: invalid index of a 0-dim tensor. Use tensor.item() to convert a 0-dim tensor to a Python number

In [9]:
torch.sigmoid(input)

TypeError: sigmoid(): argument 'input' (position 1) must be Tensor, not method